# Computing the volume underneath a Pareto frontier
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the average distance from frontier points to the model's ideal solution. This provides a measure of the conflict among the objectives.

In [2]:
import pandas as pd
import numpy as np

### Preparation for volume computation algorithm

In [166]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/2d/portfolioOptimization/portfolioOptimization.csv")
sols

,SolutionIndex,Return,Risk
0,0,1.010826,0.030845
1,1,1.010768,0.030445
2,2,1.010709,0.030045
3,3,1.010642,0.029645
4,4,1.010570,0.029245
5,5,1.010498,0.028845
6,6,1.010426,0.028445
7,7,1.010354,0.028045
8,8,1.010283,0.027645
9,9,1.010211,0.027245


In [167]:
# Hard-coded manipulations to tidy-up sols

#sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (climate change sets)
sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (pack forest and Chilean sets)
# nothing for some sets (toth mcDill, small sed fire)

In [168]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [175]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [176]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 1
#objSenses["Northern Spotted Owl Habitat (ha)"] = 1 #for climate change sets (all others 0)
#objSenses["PER"] = 0 # for tothMcDill set (all others 1)
#objSenses['MaxSediment'] = 0
objSenses[" Risk"] = 0

In [177]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [178]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [179]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [180]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [181]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N where N is number of objectives


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Computing the frontier's average distance to ideal solution

In [182]:
# Define the ideal solution

idealSolution = np.ones(len(objs))

In [183]:
# Initialize average distance to ideal

frontierAvgDist= 0

In [184]:
# Compute average distance

for solution in sols.itertuples(index=False,name=None):
    frontierAvgDist += np.linalg.norm(solution - idealSolution)
    
frontierAvgDist /= len(sols)

In [185]:
frontierAvgDist

0.69543091348300556